#### Initial setup

In [1]:
import tensorflow as tf
print(tf.__version__)

2.3.1


In [2]:
# !pip install wandb
# import wandb
import os
os.environ['WANDB_DISABLE_CODE'] = 'True'
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['WANDB_NOTEBOOK_NAME']= 'Experiments'
os.environ['WANDB_DISABLE_CODE'] = 'True'
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
# wandb.login()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Number of devices: 2


In [3]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
import losses
import random
import cv2
from imutils import paths
tf.random.set_seed(666)
np.random.seed(666)



In [ ]:

# Set of Expermimentss

# Datasets
#    1 Imagenet - 5
#    2 Cifar-10
#    3 FacesEmore - 5 
#    4 Custom - 5
    
# Losses 
#    1 Triplet loss
#    2 Max_Margin_Loss
#    3 NXTENT Loss

# Distances
#    0 None (Triplet Loss)
#    1 Cosine
#    2 Euclidian

# Models
#    0 Resnet Scratch
#    1 Facenet Pretrained

#### Datasets


##### Imagenet

In [4]:
# Train and test image paths
train_images = list(paths.list_images("Data/imagenet-5-categories/train"))
test_images = list(paths.list_images("Data/imagenet-5-categories/test"))


In [5]:
def prepare_images(image_paths):
    images = []
    labels = []

    for image in tqdm(image_paths):
        image_pixels = plt.imread(image)
        image_pixels = cv2.resize(image_pixels, (128,128))
        image_pixels = image_pixels/255.

        label = image.split("/")[3].split("_")[0]

        images.append(image_pixels)
        labels.append(label)

    images = np.array(images)
    labels = np.array(labels)

    print(images.shape, labels.shape)

    return images, labels

##### Cifar 10

In [ ]:
train_images = list(paths.list_images("Data/cifar10/train"))
test_images = list(paths.list_images("Data/cifar10/test"))

In [ ]:
def prepare_images(image_paths):
    images = []
    labels = []

    for image in tqdm(image_paths):
        image_pixels = plt.imread(image)
        image_pixels = cv2.resize(image_pixels, (128,128))
        image_pixels = image_pixels/255.

        label = image.split("/")[3]

        images.append(image_pixels)
        labels.append(label)

    images = np.array(images)
    labels = np.array(labels)

    print(images.shape, labels.shape)

    return images, labels

##### Facesemore

In [ ]:
train_images = list(paths.list_images("Data/Facesemore/"))
test_images = list(paths.list_images("Data/Facesemore/"))
# train_images = list(paths.list_images("/mnt/shared_folder/faces_emore/imgs/"))
# test_images = list(paths.list_images("Data/Facesemore/"))



In [ ]:
def prepare_images(image_paths):
    images = []
    labels = []

    for image in tqdm(image_paths):
        image_pixels = plt.imread(image)
        image_pixels = cv2.resize(image_pixels, (128,128))
        image_pixels = image_pixels/255.

        label = image.split("/")[2]

        images.append(image_pixels)
        labels.append(label)
        
    temp = list(zip(images, labels)) 
    random.shuffle(temp) 
    images, labels = zip(*temp) 

    images = np.array(images)
    labels = np.array(labels)

    print(images.shape, labels.shape)

    return images, labels

#####  Full Facesemore

In [ ]:
images = list(paths.list_images("/mnt/shared_folder/faces_emore/imgs/"))

In [ ]:
len(images)

In [ ]:
def prepare_images(image_paths):
    images = []
    labels = []

    for image in tqdm(image_paths):
        image_pixels = plt.imread(image)
        image_pixels = cv2.resize(image_pixels, (128,128))
        image_pixels = image_pixels/255.

        label = image.split("/")[4]

        images.append(image_pixels)
        labels.append(label)
        
    temp = list(zip(images, labels)) 
    random.shuffle(temp) 
    images, labels = zip(*temp) 

    images = np.array(images)
    labels = np.array(labels)

    print(images.shape, labels.shape)

    return images, labels

In [ ]:
print(images[0])
prepare_images([images[0]])

#### Dataloader

In [6]:

X_train, y_train = prepare_images(train_images)
X_test, y_test = prepare_images(test_images)


(1250, 128, 128, 3) (1250,)



(250, 128, 128, 3) (250,)


In [7]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)


In [8]:
train_ds=tf.data.Dataset.from_tensor_slices((X_train,y_train_enc))
validation_ds=tf.data.Dataset.from_tensor_slices((X_test,y_test_enc))

In [9]:
def aug(image, label):
  x=tf.image.random_brightness(image,max_delta=0)
  x=tf.image.random_contrast(x,lower=0.2, upper=1.8)
  x = tf.image.random_saturation(x, lower=0.2, upper=1.5)
  x = tf.image.random_hue(x, max_delta=0.4)
  x = tf.image.per_image_standardization(x)

  return x, label

In [10]:
BATCH_SIZE_PER_REPLICA = 64
GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [11]:
IMG_SHAPE = 128
AUTO = tf.data.experimental.AUTOTUNE
train_ds = (
    train_ds
    .shuffle(100)
#     .map(aug, num_parallel_calls=AUTO)
    .batch(GLOBAL_BATCH_SIZE)
    .prefetch(AUTO))
validation_ds = (
    validation_ds
    .shuffle(100)
    .batch(GLOBAL_BATCH_SIZE)
    .prefetch(AUTO))

In [12]:
train_dist_dataset = strategy.experimental_distribute_dataset(train_ds)
valid_dist_dataset = strategy.experimental_distribute_dataset(validation_ds)

## Model building

In [13]:
# Reference: https://github.com/wangz10/contrastive_loss/blob/master/model.py
class UnitNormLayer(tf.keras.layers.Layer):
    '''Normalize vectors (euclidean norm) in batch to unit hypersphere.
    '''
    def __init__(self):
        super(UnitNormLayer, self).__init__()

    def call(self, input_tensor):
        norm = tf.norm(input_tensor, axis=1)
        return input_tensor / tf.reshape(norm, [-1, 1])
def encoder_net():
	inputs = Input((IMG_SHAPE, IMG_SHAPE, 3))
	normalization_layer = UnitNormLayer()

	encoder = tf.keras.applications.ResNet50(weights=None, include_top=False)
	encoder.trainable = True

	embeddings = encoder(inputs, training=True)
	embeddings = GlobalAveragePooling2D()(embeddings)
	norm_embeddings = normalization_layer(embeddings)

	encoder_network = Model(inputs, norm_embeddings)

	return encoder_network
def projector_net():
	projector = tf.keras.models.Sequential([
        Dense(2048, activation="relu"),
		Dense(256, activation="relu"),
		UnitNormLayer()
	])

	return projector

In [14]:
class Resnet50_MLP(tf.keras.models.Model):
    def __init__(self):
        super(self.__class__,self).__init__()
        
        self.encoder = encoder_net()
        self.projector = projector_net()
    
    @tf.function
    def call(self, inputs):
        x = self.encoder(inputs)
        x = self.projector(x)
        return x

In [15]:
#SGD with lr decay function
decay_steps = 1000
lr_decayed_fn = tf.keras.experimental.CosineDecay(
    initial_learning_rate=0.05, decay_steps=decay_steps)
optimizer = tf.keras.optimizers.RMSprop(lr_decayed_fn)

In [29]:
# with strategy.scope():
#     def distributed_supervised_nt_xent_loss(z, y, temperature=0.5, base_temperature=0.07):
#         batch_size = tf.shape(z)[0]
#         contrast_count = 1
#         anchor_count = contrast_count
#         y = tf.expand_dims(y, -1)
#         mask = tf.cast(tf.equal(y, tf.transpose(y)), tf.float32)
#         anchor_dot_contrast = tf.divide( tf.matmul(z, tf.transpose(z)),  temperature)
#         logits_max = tf.reduce_max(anchor_dot_contrast, axis=1, keepdims=True)
#         logits = anchor_dot_contrast - logits_max
#         logits_mask = tf.ones_like(mask) - tf.eye(batch_size)
#         mask = mask * logits_mask
#         exp_logits = tf.exp(logits) * logits_mask
#         log_prob = logits -  tf.math.log(tf.reduce_sum(exp_logits, axis=1, keepdims=True))
#         # compute mean of log-likelihood over positive
#         # this may introduce NaNs due to zero division,
#         # when a class only has one example in the batch
#         mask_sum = tf.reduce_sum(mask, axis=1)
#         mean_log_prob_pos = tf.reduce_sum( mask * log_prob, axis=1)[mask_sum > 0] / mask_sum[mask_sum > 0]
#         loss = -(temperature / base_temperature) * mean_log_prob_pos
#         loss = tf.reduce_mean(loss)
#         return loss
    
#     def compute_loss(predictions, labels):
#         per_example_loss = distributed_supervised_nt_xent_loss( predictions, labels ,temperature=0.07, base_temperature=0.07)
#         print("per_example_loss: ",per_example_loss)
#         return tf.nn.compute_average_loss(per_example_loss, global_batch_size=GLOBAL_BATCH_SIZE)
    
with strategy.scope():
  # Set reduction to `none` so we can do the reduction afterwards and divide by
  # global batch size.
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True,
      reduction=tf.keras.losses.Reduction.NONE)
  def compute_loss(labels, predictions):
    per_example_loss = loss_object(labels, predictions)
    return tf.nn.compute_average_loss(per_example_loss, global_batch_size=GLOBAL_BATCH_SIZE)


In [36]:

with strategy.scope():
    model = Resnet50_MLP()

def train_step(images, labels):
    with tf.GradientTape() as tape:
        z = model(images, training=True)
        loss = compute_loss(z, labels)

    gradients = tape.gradient(loss,  model.trainable_variables)
    return loss

# @tf.function
def distributed_train_step(images, labels):
    per_replica_losses = strategy.run(train_step, args=(images, labels,))
    return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses,
                         axis=None)

In [31]:
# wandb.init(project="Experiments",id='Facesemore_sup_Resnet_augumentation')

In [33]:
for (images, labels) in train_dist_dataset:
    break

In [37]:
distributed_train_step(images, labels)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Error reported to Coordinator: Shape mismatch: The shape of labels (received (16384,)) should equal the shape of logits except for the last dimension (received (1, 64)).
Traceback (most recent call last):
  File "/home/rahulkanojia/envs/tf/lib/python3.7/site-packages/tensorflow/python/training/coordinator.py", line 297, in stop_on_exception
    yield
  File "/home/rahulkanojia/envs/tf/lib/python3.7/site-packages/tensorflow/python/distribute/mirrored_run.py", line 323, in run
    self.mai

ValueError: Shape mismatch: The shape of labels (received (16384,)) should equal the shape of logits except for the last dimension (received (1, 64)).

In [32]:
import time
EPOCHS = 300
LOG_EVERY = 10
train_loss_results = []

with strategy.scope():
    epoch_loss_avg = tf.keras.metrics.Mean(name='epoch_loss_avg')



start = time.time()
for epoch in tqdm(range(EPOCHS)):	
    
    total_loss = 0.0
    num_batches = 0
    for (images, labels) in train_dist_dataset:
        total_loss += distributed_train_step(images, labels)
        num_batches += 1
    train_loss = total_loss / num_batches
    epoch_loss_avg.update_state(train_loss) 


    train_loss_results.append(epoch_loss_avg.result())
    #wandb.log({"supervised_contrastive_loss": epoch_loss_avg.result()})

    if epoch % LOG_EVERY == 0:
        print("Epoch: {} Loss: {:.3f}".format(epoch, epoch_loss_avg.result()))

end = time.time()
# wandb.log({"training_time": end - start})

with plt.xkcd():
    plt.plot(train_loss_results)
    plt.title("Supervised Contrastive Loss")
    plt.show()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Error reported to Coordinator: in user code:

    <ipython-input-26-ddeaab43bbdb>:7 train_step  *
        loss = compute_loss(z, labels)
    <ipython-input-29-de1751ba7305>:36 compute_loss  *
        per_example_loss = loss_object(labels, predictions)
    /home/rahulkanojia/envs/tf/lib/python3.7/site-packages/tensorflow/python/keras/losses.py:149 __call__  **
        losses = ag_call(y_true, y_pred)
    /home/rahulkanojia/envs/tf/lib/python3.7/site-packages/tensorflow/python/keras/losses

ValueError: in user code:

    <ipython-input-26-ddeaab43bbdb>:14 distributed_train_step  *
        per_replica_losses = strategy.run(train_step, args=(images, labels,))
    <ipython-input-26-ddeaab43bbdb>:7 train_step  *
        loss = compute_loss(z, labels)
    <ipython-input-29-de1751ba7305>:36 compute_loss  *
        per_example_loss = loss_object(labels, predictions)
    /home/rahulkanojia/envs/tf/lib/python3.7/site-packages/tensorflow/python/keras/losses.py:149 __call__  **
        losses = ag_call(y_true, y_pred)
    /home/rahulkanojia/envs/tf/lib/python3.7/site-packages/tensorflow/python/keras/losses.py:253 call  **
        return ag_fn(y_true, y_pred, **self._fn_kwargs)
    /home/rahulkanojia/envs/tf/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:201 wrapper
        return target(*args, **kwargs)
    /home/rahulkanojia/envs/tf/lib/python3.7/site-packages/tensorflow/python/keras/losses.py:1567 sparse_categorical_crossentropy
        y_true, y_pred, from_logits=

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
def plot_embeddings(emb,labels):
  tl=TSNE()
  embedding=tl.fit_transform(emb)
  fig = plt.figure(figsize = (10, 10))
  sns.scatterplot(embedding[:,0], embedding[:,1], hue=labels)
  plt.show()

  return fig

encoded_vector=encoder_r.predict(X_train)
fig = plot_embeddings(encoded_vector,y_train)

In [ ]:
# wandb.log(({"facesemore contrastive loss Augumentation": wandb.Image(fig)}))

In [ ]:
def supervised_model():
	inputs = Input((IMG_SHAPE, IMG_SHAPE, 3))
	encoder_r.trainable = False

	r = encoder_r(inputs, training=False)
	outputs = Dense(5, activation='softmax')(r)

	supervised_model = Model(inputs, outputs)
  
	return supervised_model

In [ ]:
from wandb.keras import WandbCallback

In [ ]:
optimizer2 = tf.keras.optimizers.RMSprop(learning_rate=1e-3)

In [ ]:
supervised_classifier = supervised_model()

supervised_classifier.compile(optimizer=optimizer2,
	loss=tf.keras.losses.SparseCategoricalCrossentropy(),
	metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# es = tf.keras.callbacks.EarlyStopping(monitor="val_sparse_categorical_accuracy", patience=2,
# 	restore_best_weights=True, verbose=2)

# wandb.init(project="scl", entity="authors", id='FSL_RMS_LRDECAY ')

start = time.time()
supervised_classifier.fit(train_ds,
	validation_data=validation_ds,
	epochs=200)
# 	callbacks=[es])
end = time.time()


In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
def plot_embeddings(emb,labels):
  tl=TSNE()
  embedding=tl.fit_transform(emb)
  fig = plt.figure(figsize = (10, 10))
  sns.scatterplot(embedding[:,0], embedding[:,1], hue=labels)
  plt.show()

  return fig

encoded_vector=supervised_classifier.predict(X_train)
# fig = plot_embeddings(encoded_vector,y_train)

In [ ]:
for i in range(30):
    print(i, np.argmax(encoded_vector[i]))

In [ ]:
y_test